In [3]:
import json
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

def extract_id(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    result = []

    for item in data:
        if 'id' in item:
            result.append(item['id'])

    return result

def download_info(id):
    try:
        with webdriver.Chrome() as driver:
            driver.get(f'https://m.place.naver.com/place/{id}/home')
            driver.implicitly_wait(5)

            elements_to_click = [
                '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div.O8qbU.pSavy > div > a > div > div > span > svg',
                '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(8) > div > a > span.rvCSr > svg'
            ]

            for element_selector in elements_to_click:
                try:
                    elements = driver.find_elements(By.CSS_SELECTOR, element_selector)
                    if elements:
                        elements[0].click()
                        driver.implicitly_wait(5)

                except Exception as e:
                    print(f'Error: {e}')
                    pass

            hospital_info_element = []

            all_elements = driver.find_elements(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div *')
            for element in all_elements:
                hospital_info_element.append(element.text)

            return hospital_info_element

    except Exception as e:
        print(f'Error: {e}')
        return None

json_file_path = 'naver_total.json'
extracted_data = extract_id(json_file_path)

extracted_data = extracted_data[:10]

with Pool() as pool:
    results = list(tqdm(pool.imap(download_info, extracted_data), total=len(extracted_data), desc='Downloading info'))

hospital_info = [info for info in results if info is not None]

print(hospital_info)